##数据预处理

In [50]:
import pandas as pd

# 读取 Excel 文件
file1_path = '../附件1.xlsx'
file2_path = '../附件2.xlsx'

# 读取所有sheet
land_data = pd.read_excel(file1_path, sheet_name='乡村的现有耕地')  # 附件1中包含地块信息
land_crop_data = pd.read_excel(file1_path, sheet_name='乡村种植的农作物')  # 附件1中种植信息

crop_planting_data = pd.read_excel(file2_path, sheet_name='2023年的农作物种植情况')  # 附件2中2023年作物种植情况
crop_stats_data = pd.read_excel(file2_path, sheet_name='2023年统计的相关数据')  # 附件2中2023年的相关数据

# 定义地块前缀对应的地块类型映射
land_type_mapping = {
    'A': '平旱地',
    'B': '梯田',
    'C': '山坡地',
    'D': '水浇地',
    'E': '普通大棚 ',
    'F': '智慧大棚'
   
}

# 从地块名称中提取前缀并映射到地块类型
land_data = land_data.drop('说明 ', axis=1)
crop_stats_data = crop_stats_data.drop(['序号', '作物编号'], axis=1)
land_data['地块前缀'] = land_data['地块名称'].str[0]  # 提取第一个字符作为前缀
land_data['地块类型'] = land_data['地块前缀'].map(land_type_mapping)  # 映射到地块类型

# 合并地块信息和种植信息（基于地块类型）
merged_data = land_data.merge(crop_stats_data, on='地块类型', how='left')

# 根据种植季次生成季节限制
def generate_season_restriction(row):
    # 如果种植季次是"单季"，季节限制设为1
    if row['种植季次'] == '单季':
        row['季节限制'] = 1
        row['第一季单价(元)'] = row['销售单价/(元/斤)']
        row['第二季单价(元)'] = 0
    # 如果种植季次是"第一季"或"第二季"，季节限制设为2（表示可种两季）
    elif row['种植季次'] in ['第一季', '第二季']:
        row['季节限制'] = 2
        if row['种植季次'] == '第一季':
            row['第一季单价(元)'] = row['销售单价/(元/斤)']
        else:
            row['第二季单价(元)'] = row['销售单价/(元/斤)']
            
    return row

# 应用生成季节限制的逻辑
merged_data = merged_data.apply(generate_season_restriction, axis=1)
merged_data =merged_data.fillna(0)
df=merged_data 
#处理单价区间，假设销售价格服从正态分布，其期望为均值
df['第一季最小单价(元)']=df['第一季单价(元)'].str.split('-', expand=True)[0].astype(float, errors='ignore')
df['第一季最大单价(元)']=df['第一季单价(元)'].str.split('-', expand=True)[1].astype(float, errors='ignore')
df['第二季最小单价(元)']=df['第二季单价(元)'].str.split('-', expand=True)[0].astype(float, errors='ignore')
df['第二季最大单价(元)']=df['第二季单价(元)'].str.split('-', expand=True)[1].astype(float, errors='ignore')
df=df.fillna(0)

#2023产量
crop_recom=pd.read_excel("../crop_commd.xlsx")

##模型所需数据结构化及变量定义

In [51]:

# 将DataFrame转化为优化模型需要的字典格式
data = {}
for idx, row in df.iterrows():
    data[(row['地块名称'], row['作物名称'])] = (
        row['地块类型'],
        row['亩产量/斤'], 
        row['种植成本/(元/亩)'],
        (row['第一季最小单价(元)']+row['第一季最大单价(元)'])/2,
        (row['第二季最小单价(元)']+row['第二季最大单价(元)'])/2,
        row['季节限制']
    )

# 地块和作物名称
land_plots = df['地块名称'].unique().tolist()
crops = df['作物名称'].unique().tolist()
land_area = df.set_index('地块名称')['地块面积/亩'].to_dict()
# 定义豆类作物的列表
legumes = ['黄豆', '黑豆','红豆','绿豆','爬豆','豇豆','刀豆','芸豆',]  # 豆类作物
years = [2024, 2025, 2026, 2027, 2028, 2029, 2030]  # 年份

# 市场需求，初始可以假设为2023年产量
crop_demand = {}
for idx, row in crop_recom.iterrows():
    crop_demand[row['作物名称']] = row['总产量'] 

##辅助函数定义

In [52]:

# 辅助函数：判断是否为有效的地块和作物的季节种植组合
def is_valid_plot(i, j, s):
    # 检查地块类型和作物季节限制
    plot_type = data[(i, j)][0]  # 地块类型
    if plot_type in ['平旱地', '梯田', '山坡地'] and s == 2:
        return False  # 平旱地、梯田和山坡地只能种一季
    if plot_type == '水浇地' and s == 1 and j == '蔬菜':
        return False  # 水浇地第一季不能种蔬菜
    if plot_type == '普通大棚' and j == '水稻':
        return False  # 普通大棚不能种水稻
    return True


##情形一：超出部分无收益

In [53]:

import pulp as lp
# 决策变量：x[i][j][s][y] 表示在第 i 块地上种植第 j 种作物的面积，s 表示季节 (1 或 2)，y 表示年份
x = lp.LpVariable.dicts(
    "x", 
    ((i, j, s, y) for i in land_plots for j in crops for s in [1, 2] for y in years
     if (i, j) in data and (s == 1 or data[(i, j)][5] == 2) and data[(i, j)][3 if s == 1 else 4] != 0), 
    lowBound=0, cat="Continuous"
)

#创建问题
prob = lp.LpProblem("Crop_Planting_Optimization", lp.LpMaximize)

# 对于每个三年时间窗口，确保豆类作物的种植面积达到总地块面积的 100%

#第一阶段目标优化
# 确保每块地在3年内的所有面积都至少种植一次豆类
for i in land_plots:
    for y in range(2023, max(years) - 4):  # 遍历每个连续的三年窗口
        prob += lp.lpSum(
            x[(i, j, s, y+k)] for j in legumes for s in [1, 2] for k in range(3)
            if (i, j, s, y+k) in x and (s == 1 or data[(i, j)][5] == 2)  # 只遍历允许的季节
        ) >= land_area[i], f"Legume_Rotation_{i}_{y}"  # 设置为总面积

# 定义 waste 变量，表示超过产量上限的部分
waste = lp.LpVariable.dicts("waste", ((j, y) for j in crops for y in years), lowBound=0, cat="Continuous")
min_area_thresholdd=0

# 去除作物名称中的空格
for j in crops:
    j_clean = j.strip()  # 去掉作物名称前后的空格
    for y in years:
        for i in land_plots:
            for s in [1, 2]:  # 添加季节 s 来确保约束名称唯一
                # 添加条件到 for 循环中，确保只处理符合条件的组合
                prob += lp.lpSum(
                    x.get((i, j_clean, s, y), 0) * data.get((i, j_clean), [0, 0])[1]
                    for i in land_plots if (i, j_clean) in data and (s == 1 or data[(i, j_clean)][5] == 2)
                ) <= crop_demand.get(j_clean, 0) + waste.get((j_clean, y), 0), \
                f"Crop_Yield_Limit_With_Waste_{i}_{j_clean}_{y}_S{s}"  # 加上季节 s，确保唯一


#第二阶段目标优化
# 目标函数，扣除滞销部分
prob += lp.lpSum(
    (x[(i, j, s, y)] * data[(i, j)][1] * data[(i, j)][3 if s == 1 else 4]  # 正常销售的收益
     - data[(i, j)][2] * x[(i, j, s, y)]  # 减去种植成本
     - waste[j, y] * data[(i, j)][3 if s == 1 else 4])  # 滞销部分不产生收益
    for i in land_plots for j in crops for s in [1, 2] for y in years
    if (i, j) in data 
    and (s == 1 or data[(i, j)][5] == 2) 
    and data[(i, j)][3 if s == 1 else 4] != 0 
    and is_valid_plot(i, j, s)
), "Maximize_Total_Profit_With_Waste"


# 约束1：每块地的总种植面积不能超过可用面积
for i in land_plots:
    for y in years:  # 遍历每个年份
        prob += lp.lpSum(x[(i, j, s, y)] for j in crops for s in [1, 2] 
                         if (i, j) in data and (s == 1 or data[(i, j)][5] == 2) 
                         and (i, j, s, y) in x) <= land_area[i], f"Land_Area_Constraint_{i}_{y}"

 
#约束条件2：种植不能太分散,方便管理
for j in crops:
    for i in land_plots:
        for y in years:
            # 根据地块面积动态调整最小种植面积
            if land_area[i] > 10:  # 面积大于10亩的地块
                min_area_threshold = 0.1 * land_area[i]  # 地块总面积的10%
            else:  # 面积较小的地块
                min_area_threshold = 0.05 * land_area[i]  # 地块总面积的5%
            
            # 确保每个约束的名称唯一，添加季节(s)变量到约束名中
            for s in [1, 2]:
                prob += lp.lpSum(x[(i, j, s, y)] for s in [1, 2] if (i, j, s, y) in x) >= min_area_thresholdd, \
                        f"Min_Concentrated_Area_Constraint_{i}_{j}_{y}_S{s}"
                       
#约束3：禁止同一地块同一年在两季内种植同一作物
for i in land_plots:
    for j in crops:
        for y in years:  # 遍历每一年
            if (i, j, 1, y) in x and (i, j, 2, y) in x:  # 检查变量是否存在
                prob += x[(i, j, 1, y)] + x[(i, j, 2, y)] <= land_area[i], f"No_Same_Crop_Two_Seasons_{i}_{j}_Year_{y}"

#约束4：禁止连续两年种植同一种作物（防止重茬)
for i in land_plots:
    for j in crops:
        for y in years[:-1]:  # 遍历年份，检查连续两年的种植情况
            # 第一季重茬限制
            if (i, j, 1, y) in x and (i, j, 1, y+1) in x:
                prob += x[(i, j, 1, y)] + x[(i, j, 1, y+1)] <= land_area[i], f"No_Continuous_Planting_{i}_{j}_{y}_S1"
            # 第二季重茬限制
            if (i, j, 2, y) in x and (i, j, 2, y+1) in x:
                prob += x[(i, j, 2, y)] + x[(i, j, 2, y+1)] <= land_area[i], f"No_Continuous_Planting_{i}_{j}_{y}_S2"


# 求解问题
prob.solve()

# 输出求解状态
print(f"状态: {lp.LpStatus[prob.status]}")

# 按年份输出种植决策和当年利润
for y in years:
    print(f"\n=====情形一：{y} 年的种植决策 =====")
    
    yearly_profit = 0  # 初始化当年的利润
    
    for i in land_plots:
        for j in crops:
            for s in [1, 2]:
                if (i, j, s, y) in x:  # 确保变量存在
                    planted_area = x[(i, j, s, y)].varValue
                    if planted_area > 0:  # 只显示种植面积大于0的作物
                        print(f"地块 {i} 第{s}季 种植 {j} {planted_area:.2f} 亩")
                        # 计算该作物的利润并累加
                        yearly_profit += (
                            planted_area * data[(i, j)][1] * data[(i, j)][3 if s == 1 else 4]  # 种植面积 * 产量 * 单价
                            - data[(i, j)][2] * planted_area  # 减去种植成本
                        )
    
    # 输出当年的利润
    print(f"\n{y} 年的利润: {yearly_profit:.2f} 元")



'\n# 决策变量：x[i][j][s][y] 表示在第 i 块地上种植第 j 种作物的面积，s 表示季节 (1 或 2)，y 表示年份\nx = lp.LpVariable.dicts(\n    "x", \n    ((i, j, s, y) for i in land_plots for j in crops for s in [1, 2] for y in years\n     if (i, j) in data and (s == 1 or data[(i, j)][5] == 2) and data[(i, j)][3 if s == 1 else 4] != 0), \n    lowBound=0, cat="Continuous"\n)\nprob = lp.LpProblem("Crop_Planting_Optimization", lp.LpMaximize)\n# 对于每个三年时间窗口，确保豆类作物的种植面积达到总地块面积的 100%\n#第一阶段目标优化\n# 确保每块地在3年内的所有面积都至少种植一次豆类\nfor i in land_plots:\n    for y in range(2023, max(years) - 4):  # 遍历每个连续的三年窗口\n        prob += lp.lpSum(\n            x[(i, j, s, y+k)] for j in legumes for s in [1, 2] for k in range(3)\n            if (i, j, s, y+k) in x and (s == 1 or data[(i, j)][5] == 2)  # 只遍历允许的季节\n        ) >= land_area[i], f"Legume_Rotation_{i}_{y}"  # 设置为总面积\n# 定义 waste 变量，表示超过产量上限的部分\nwaste = lp.LpVariable.dicts("waste", ((j, y) for j in crops for y in years), lowBound=0, cat="Continuous")\nmin_area_thresholdd=0\n# 去除作物名称中的空格\nfor j in cr

##情形二：超出部分按50%售价处理

In [54]:
import pulp as lp
# 决策变量：x[i][j][s][y] 表示在第 i 块地上种植第 j 种作物的面积，s 表示季节 (1 或 2)，y 表示年份
x = lp.LpVariable.dicts(
    "x", 
    ((i, j, s, y) for i in land_plots for j in crops for s in [1, 2] for y in years
     if (i, j) in data and (s == 1 or data[(i, j)][5] == 2) and data[(i, j)][3 if s == 1 else 4] != 0), 
    lowBound=0, cat="Continuous"
)

#创建问题
prob = lp.LpProblem("Crop_Planting_Optimization", lp.LpMaximize)
min_area_thresholdd=0


#第一阶段目标优化
# 确保每块地在3年内的所有面积都至少种植一次豆类
for i in land_plots:
    for y in range(2023, max(years) - 4):  # 遍历每个连续的三年窗口
        prob += lp.lpSum(
            x[(i, j, s, y+k)] for j in legumes for s in [1, 2] for k in range(3)
            if (i, j, s, y+k) in x and (s == 1 or data[(i, j)][5] == 2)  # 只遍历允许的季节
        ) >= land_area[i], f"Legume_Rotation_{i}_{y}"  # 设置为总面积

for i in land_plots:
    for y in years:  # 遍历每个年份
        prob += lp.lpSum(x[(i, j, s, y)] for j in crops for s in [1, 2] 
                         if (i, j) in data and (s == 1 or data[(i, j)][5] == 2) 
                         and (i, j, s, y) in x) <= land_area[i], f"Land_Area_Constraint_{i}_{y}"
        

# 定义 waste 变量，去掉作物名称的空格
waste = lp.LpVariable.dicts("waste", ((j.strip(), y) for j in crops for y in years), lowBound=0, cat="Continuous")
# 去除作物名称中的空格
for j in crops:
    j_clean = j.strip()  # 去掉作物名称前后的空格
    for y in years:
        for i in land_plots:
            for s in [1, 2]:  # 添加季节 s 来确保约束名称唯一
                # 添加条件到 for 循环中，确保只处理符合条件的组合
                prob += lp.lpSum(
                    x.get((i, j_clean, s, y), 0) * data.get((i, j_clean), [0, 0])[1]
                    for i in land_plots if (i, j_clean) in data and (s == 1 or data[(i, j_clean)][5] == 2)
                ) <= crop_demand.get(j_clean, 0) + waste.get((j_clean, y), 0), \
                f"Crop_Yield_Limit_With_Waste_{i}_{j_clean}_{y}_S{s}"  # 加上季节 s，确保唯一
#第二阶段目标优化
prob += lp.lpSum(
    (x[(i, j, s, y)] * data[(i, j)][1] * data[(i, j)][3 if s == 1 else 4]  # 正常销售的收益
     - data[(i, j)][2] * x[(i, j, s, y)]  # 减去种植成本
     - waste.get((j, y), 0) * 0.5 * data[(i, j)][3 if s == 1 else 4])  # 超过部分按50%价格出售 
    for i in land_plots for j in crops for s in [1, 2] for y in years
    if (i, j) in data 
    and (s == 1 or data[(i, j)][5] == 2) 
    and data[(i, j)][3 if s == 1 else 4] != 0 
    and is_valid_plot(i, j, s)
), "Maximize_Total_Profit_With_Discounted_Sales"

# 约束1：每块地的总种植面积不能超过可用面积
# 修改 Land_Area_Constraint 名称，添加季节 s
for i in land_plots:
    for y in years:  # 遍历每个年份
        for s in [1, 2]:  # 添加季节
            prob += lp.lpSum(x[(i, j, s, y)] for j in crops  # 获取种植面积
                             if (i, j) in data and (s == 1 or data[(i, j)][5] == 2) 
                             and (i, j, s, y) in x) <= land_area[i], \
                             f"Land_Area_Constraint_{i}_{y}_S{s}"

#约束条件2：种植不能太分散
for j in crops:
    for i in land_plots:
        for y in years:
            # 根据地块面积动态调整最小种植面积
            if land_area[i] > 10:  # 面积大于10亩的地块
                min_area_threshold = 0.1 * land_area[i]  # 地块总面积的10%
            else:  # 面积较小的地块
                min_area_threshold = 0.05 * land_area[i]  # 地块总面积的5%
            
            # 确保每个约束的名称唯一，添加季节(s)变量到约束名中
            for s in [1, 2]:
                prob += lp.lpSum(x[(i, j, s, y)] for s in [1, 2] if (i, j, s, y) in x) >= min_area_thresholdd, \
                        f"Min_Concentrated_Area_Constraint_{i}_{j}_{y}_S{s}"
       
#约束3：禁止同一年内同一地块种植同一作物在两季
for i in land_plots:
    for j in crops:
        for y in years:  # 遍历每一年
            if (i, j, 1, y) in x and (i, j, 2, y) in x:  # 检查变量是否存在
                prob += x[(i, j, 1, y)] + x[(i, j, 2, y)] <= land_area[i], f"No_Same_Crop_Two_Seasons_{i}_{j}_Year_{y}"

#约束4：禁止连续两年种植同一种作物（防止重茬)
for i in land_plots:
    for j in crops:
        for y in years[:-1]:  # 遍历年份，检查连续两年的种植情况
            # 第一季重茬限制
            if (i, j, 1, y) in x and (i, j, 1, y+1) in x:
                prob += x[(i, j, 1, y)] + x[(i, j, 1, y+1)] <= land_area[i], f"No_Continuous_Planting_{i}_{j}_{y}_S1"
            # 第二季重茬限制
            if (i, j, 2, y) in x and (i, j, 2, y+1) in x:
                prob += x[(i, j, 2, y)] + x[(i, j, 2, y+1)] <= land_area[i], f"No_Continuous_Planting_{i}_{j}_{y}_S2"


# 求解问题
prob.solve()

# 输出求解状态
print(f"状态: {lp.LpStatus[prob.status]}")

# 按年份输出种植决策和当年利润
for y in years:
    print(f"\n===== 情形二：{y} 年的种植决策 =====")
    
    yearly_profit = 0  # 初始化当年的利润
    
    for i in land_plots:
        for j in crops:
            for s in [1, 2]:
                if (i, j, s, y) in x:  # 确保变量存在
                    planted_area = x[(i, j, s, y)].varValue
                    if planted_area > 0:  # 只显示种植面积大于0的作物
                        print(f"地块 {i} 第{s}季 种植 {j} {planted_area:.2f} 亩")
                        # 计算该作物的利润并累加
                        yearly_profit += (
                            planted_area * data[(i, j)][1] * data[(i, j)][3 if s == 1 else 4]  # 种植面积 * 产量 * 单价
                            - data[(i, j)][2] * planted_area  # 减去种植成本
                        )
    
    # 输出当年的利润
    print(f"\n{y} 年的利润: {yearly_profit:.2f} 元")



状态: Optimal

===== 情形二：2024 年的种植决策 =====
地块 A1 第1季 种植 黄豆 0.00 亩
地块 A1 第1季 种植 小麦 51.94 亩
地块 A1 第1季 种植 南瓜 11.70 亩
地块 A1 第1季 种植 红薯 16.36 亩
地块 A2 第1季 种植 黄豆 1.88 亩
地块 A2 第1季 种植 玉米 53.12 亩
地块 A3 第1季 种植 黄豆 21.29 亩
地块 A3 第1季 种植 玉米 13.71 亩
地块 A4 第1季 种植 黄豆 4.62 亩
地块 A4 第1季 种植 小麦 17.82 亩
地块 A4 第1季 种植 玉米 49.55 亩
地块 A5 第1季 种植 黄豆 68.00 亩
地块 A5 第1季 种植 玉米 0.00 亩
地块 A6 第1季 种植 黄豆 14.84 亩
地块 A6 第1季 种植 爬豆 23.80 亩
地块 A6 第1季 种植 玉米 16.36 亩
地块 B1 第1季 种植 黄豆 58.95 亩
地块 B1 第1季 种植 小麦 1.05 亩
地块 B2 第1季 种植 小麦 46.00 亩
地块 B2 第1季 种植 谷子 0.00 亩
地块 B3 第1季 种植 绿豆 24.94 亩
地块 B3 第1季 种植 小麦 15.06 亩
地块 B4 第1季 种植 黄豆 28.00 亩
地块 B5 第1季 种植 小麦 25.00 亩
地块 B5 第1季 种植 谷子 0.00 亩
地块 B6 第1季 种植 黑豆 46.00 亩
地块 B6 第1季 种植 绿豆 11.03 亩
地块 B6 第1季 种植 谷子 28.97 亩
地块 B7 第1季 种植 黄豆 55.00 亩
地块 B7 第1季 种植 小麦 0.00 亩
地块 B8 第1季 种植 黄豆 36.32 亩
地块 B8 第1季 种植 高粱 7.68 亩
地块 B9 第1季 种植 黄豆 17.26 亩
地块 B9 第1季 种植 红豆 0.26 亩
地块 B9 第1季 种植 绿豆 18.22 亩
地块 B9 第1季 种植 小麦 14.26 亩
地块 B9 第1季 种植 高粱 0.00 亩
地块 B10 第1季 种植 绿豆 10.94 亩
地块 B10 第1季 种植 小麦 14.06 亩
地块 B11 第1季 种植 黄豆 15.03 亩
地块 B11 

##将结果写入文件中

In [55]:
# 打开一个记事本文件进行写入（如果文件不存在会自动创建）
with open("result_21.txt", "w", encoding="utf-8") as file:
    for y in years:
        # 写入年份的标题
        file.write(f"\n===== {y} 年的种植决策 =====\n")
        
        yearly_profit = 0  # 初始化当年的利润
        
        for i in land_plots:
            for j in crops:
                for s in [1, 2]:
                    if (i, j, s, y) in x:  # 确保变量存在
                        planted_area = x[(i, j, s, y)].varValue
                        if planted_area > 0:  # 只记录种植面积大于0的作物
                            # 写入每个地块、作物的种植情况
                            file.write(f"地块 {i} 第{s}季 种植 {j} {planted_area:.2f} 亩\n")
                            # 计算该作物的利润并累加
                            yearly_profit += (
                                planted_area * data[(i, j)][1] * data[(i, j)][3 if s == 1 else 4]  # 种植面积 * 产量 * 单价
                                - data[(i, j)][2] * planted_area  # 减去种植成本
                            )
        
        # 写入当年的利润
        file.write(f"\n{y} 年的利润: {yearly_profit:.2f} 元\n")
